In [1]:
# Pip install necessary libraries
#%pip install echopype s3fs boto3==1.34.51 numpy==1.24.4 xarray==2022.12.0

In [2]:
# Import necessary libraries
import s3fs
import boto3
import echopype as ep
from botocore import UNSIGNED
from botocore.config import Config

In [3]:
# Grab from S3 and parse EK60 
s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED))
s3_file_system = s3fs.S3FileSystem(anon=True)

bucket_name = 'noaa-wcsd-pds'
ship_name = 'Albatross_Iv'
cruise_name = 'AL0403'
sensor_name = 'EK60'
file_name = "L0010-D20040416-T094042-EK60.raw"

raw_file_s3_path = f"s3://{bucket_name}/data/raw/{ship_name}/{cruise_name}/{sensor_name}/{file_name}"
echodata = ep.open_raw(raw_file_s3_path, sonar_model=sensor_name, use_swap=True, storage_options={'anon': True})

In [4]:
# Turn on Echopype Verbosity
ep.utils.log.verbose(override=False)

# Print out pre interpolated latitudes
latitude = echodata.platform.latitude.values
latitude_values = latitude.copy()
latitude_values.sort()
print("Latitude Values Pre Interp (lowest 5):", latitude_values[:5])

ds_sv = ep.calibrate.compute_Sv(echodata)
ds_sv_location = ep.consolidate.add_location(ds_sv, echodata)

# Print out post interpolated latitudes
latitude_2 = ds_sv_location.latitude.values
latitude_values_2 = latitude_2.copy()
latitude_values_2.sort()
print("Latitude Values Post Interp (lowest 5):", latitude_values_2[:5])

Latitude Values Pre Interp (lowest 5): [ 0.          0.         43.68361    43.68362333 43.68362333]


2024-04-04 22:09:35,389:echopype.consolidate.api:WARNING: The echodata["Platform"]["latitude"] array contains zeros. Interpolation may be negatively impacted, so the user should handle these values.
2024-04-04 22:09:35,390:echopype.consolidate.api:WARNING: The echodata["Platform"]["longitude"] array contains zeros. Interpolation may be negatively impacted, so the user should handle these values.


Latitude Values Post Interp (lowest 5): [19.57221436 23.97964969 43.68361412 43.68363238 43.68365699]
